In [1]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_amanda_dynamic
from sklearn.model_selection import ParameterGrid



def writeResults(datasetID, bestScore, bestParams, clfName, distanceMetric, beta=None):
    if distanceMetric == 'BBD' and isinstance(beta,list):
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_EVL-{}-ALLBETAS.txt".format(distanceMetric,clfName)
    elif distanceMetric == 'BBD':
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}{:.3f}_EVL-{}.txt".format(distanceMetric, beta,clfName)
    else:
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_EVL-{}.txt".format(distanceMetric,clfName)
        
    file = open(path,"a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main():
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading datasets
    datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, 
                setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
                setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
                setup.loadElecData, setup.loadKeystroke, setup.loadNOAADataset]
    
    #arrClfName = ['SGD', 'NB', 'RF', 'LP', 'KNN']
    arrClfName = ['LP']
    distanceMetric = 'BBD'
    beta = [-50,-10, -5, -2, -1, -0.5, -0.1, -0.01, -0.001, 1.001, 1.01, 1.1, 1.5, 2, 3, 5, 10, 50]
    allBetas=True
    for clfName in arrClfName:
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("**************** BEGIN of {}-{}{:.3f} results ****************".format(clfName, distanceMetric, beta))            
        else:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        batches=3
        poolSize = None
        isBatchMode = True
        #testing grid search
        for i in range(len(datasets)):
            finalScore = 0
            best_grid={}
            dataValues, dataLabels, description = datasets[i](path, sep)

            #Train-test split
            availableQty = int(0.05*len(dataLabels))
            availableLabels = dataLabels[:availableQty] 
            availableData = dataValues[:availableQty]

            # 70/30 train/test data
            initialLabeledData = int(0.7*len(availableLabels))
            sizeOfBatch = int((len(availableLabels)-initialLabeledData)/batches)

            print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))

            tuned_params = [{"sizeOfBatch":[sizeOfBatch], 
                             "batches":[batches], "poolSize":[poolSize], "isBatchMode":[isBatchMode], 
                             "initialLabeledData":[initialLabeledData], "clfName":[clfName],
                            "distanceMetric":[distanceMetric], "beta":beta}]
            if clfName == 'LP' or clfName == 'KNN':
                tuned_params[0].update({"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})

            for g in ParameterGrid(tuned_params):
                averageAccuracy=0
                gs = grid_selection_amanda_dynamic.run(**g)

                try:
                    gs.fit(availableData, availableLabels)
                    averageAccuracy = np.mean(gs.predict())
                    print(averageAccuracy, g)
                    if finalScore < averageAccuracy:
                        finalScore = averageAccuracy
                        best_grid = g
                except Exception:
                    print("An error occured in ", description, g)
                    #raise Exception

            print(finalScore)
            print(best_grid)
            print("=======================================================================================================")

            writeResults(description, finalScore, best_grid, clfName, distanceMetric, beta)
        
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("******** END of {}-{}{:.3f} results ********".format(clfName, distanceMetric, beta))
        else:
            print("******** END of {}-{} results ********".format(clfName, distanceMetric))
    
if __name__ == "__main__":
    main()


**************** BEGIN of LP-BBD results ****************
One Class Diagonal Translation. 2 Dimensional data.: 3 batches of 80 instances
98.75 {'K': 2, 'batches': 3, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
98.75 {'K': 2, 'batches': 3, 'beta': -10, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
98.75 {'K': 2, 'batches': 3, 'beta': -5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
98.75 {'K': 2, 'batches': 3, 'beta': -2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.16666666666667 {'K': 2, 'batches': 3, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.166666

C:\Users\bruno.silva\Desktop\TCC\AMANDA\setup.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  dataValues = pd.DataFrame.as_matrix(dataValues)


99.16666666666667 {'K': 3, 'batches': 3, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.16666666666667 {'K': 3, 'batches': 3, 'beta': -10, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.16666666666667 {'K': 3, 'batches': 3, 'beta': -5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.16666666666667 {'K': 3, 'batches': 3, 'beta': -2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.58333333333333 {'K': 3, 'batches': 3, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.58333333333333 {'K': 3, 'batches': 3, 'beta': -0.5, 'clfName': 'LP', 'distanceMetric': 'BBD', '

99.16666666666667 {'K': 5, 'batches': 3, 'beta': 10, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.16666666666667 {'K': 5, 'batches': 3, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.16666666666667 {'K': 6, 'batches': 3, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.16666666666667 {'K': 6, 'batches': 3, 'beta': -10, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.16666666666667 {'K': 6, 'batches': 3, 'beta': -5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.16666666666667 {'K': 6, 'batches': 3, 'beta': -2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'in

99.58333333333333 {'K': 8, 'batches': 3, 'beta': 3, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.58333333333333 {'K': 8, 'batches': 3, 'beta': 5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.58333333333333 {'K': 8, 'batches': 3, 'beta': 10, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.58333333333333 {'K': 8, 'batches': 3, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.58333333333333 {'K': 9, 'batches': 3, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 560, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 80}
99.58333333333333 {'K': 9, 'batches': 3, 'beta': -10, 'clfName': 'LP', 'distanceMetric': 'BBD', 'init

...

C:\Users\bruno.silva\Desktop\TCC\AMANDA\setup.py:281: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  dataValues = pd.DataFrame.as_matrix(dataValues)


76.92333333333333 {'K': 2, 'batches': 3, 'beta': -0.1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
76.92333333333333 {'K': 2, 'batches': 3, 'beta': -0.01, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
76.92333333333333 {'K': 2, 'batches': 3, 'beta': -0.001, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
79.48666666666666 {'K': 2, 'batches': 3, 'beta': 1.001, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
79.48666666666666 {'K': 2, 'batches': 3, 'beta': 1.01, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
79.48666666666666 {'K': 2, 'batches': 3, 'beta': 1.1, 'clfName': 'LP', 'distanceMetric

74.35666666666667 {'K': 5, 'batches': 3, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
73.62666666666667 {'K': 5, 'batches': 3, 'beta': -0.5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
71.06 {'K': 5, 'batches': 3, 'beta': -0.1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
71.06 {'K': 5, 'batches': 3, 'beta': -0.01, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
71.06 {'K': 5, 'batches': 3, 'beta': -0.001, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
73.62666666666667 {'K': 5, 'batches': 3, 'beta': 1.001, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 

78.38666666666667 {'K': 7, 'batches': 3, 'beta': 5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
78.38666666666667 {'K': 7, 'batches': 3, 'beta': 10, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
78.38666666666667 {'K': 7, 'batches': 3, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
71.42666666666666 {'K': 8, 'batches': 3, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
71.42666666666666 {'K': 8, 'batches': 3, 'beta': -10, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
71.42666666666666 {'K': 8, 'batches': 3, 'beta': -5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'ini

71.42666666666666 {'K': 10, 'batches': 3, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
71.06 {'K': 11, 'batches': 3, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
71.06 {'K': 11, 'batches': 3, 'beta': -10, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
71.06 {'K': 11, 'batches': 3, 'beta': -5, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
70.69333333333334 {'K': 11, 'batches': 3, 'beta': -2, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 91}
71.06 {'K': 11, 'batches': 3, 'beta': -1, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 634, 'isBatchMode': Tru